In [1]:
from wequant import *
from wequant.wefetch import *
from wequant.wefetch.query import *
from wequant.wefetch.query_ad import *
from wequant.mongo import get_db
import pandas as pd
import os
import pandas as pd 
import numpy as np

In [26]:
fetch_stock_dk_adv("000001.SZ",'2020-01-01','2027-01-01').to_qfq()

,,trade_date,L_bias,L_label,L_switch,L_value,R_bias,R_label,R_switch,R_value,acitve1,acitve2,base_code,datetime,exceed1,exceed2,exceed3,switch1,switch2,switch3,type
date,code,,,,,,,,,,,,,,,,,,,,
2026-02-02,000001.SZ,20260202,0.0,1,0,11.065,0.0,0,0,12.007,1,0,000001,20260202,0,0,0,0.013,0,0,stock


In [27]:
fetch_index_dk("000001.SH",'2020-01-01','2027-01-01',format='pd')

,code,trade_date,L_bias,L_label,L_switch,L_value,R_bias,R_label,R_switch,R_value,...,base_code,datetime,exceed1,exceed2,exceed3,switch1,switch2,switch3,type,date
0,000001.SH,20260202,5.667,0,0,3788.166,0.0,0,0,4083.925,...,000001,20260202,0,0,0,0.0,0,0,index,2026-02-02


In [20]:
fetch_future_dk("SCL8",'2020-01-01','2027-01-01')


array([['SCL8', '20260202', 6.138, 0, 0, 423.318, 0.0, 0, 0, 491.743, 0,
        2, 'SCL8', '20260202', 0, -1, 0, -2.663, 0, -1, 'future',
        Timestamp('2026-02-02 00:00:00')]], dtype=object)

In [7]:
fetch_hkstock_day("09988",'2020-01-01','2026-01-01')

array([['09988', 208.0000152588, 211.0000152588, ..., 951, 1999240704.0,
        Timestamp('2020-01-02 00:00:00')],
       ['09988', 215.0000152588, 215.0000152588, ..., 1130, 2407183872.0,
        Timestamp('2020-01-03 00:00:00')],
       ['09988', 211.0000152588, 211.8000030518, ..., 1130, 2381117952.0,
        Timestamp('2020-01-06 00:00:00')],
       ...,
       ['09988', 154.0, 158.0, ..., 11086, 17256390656.0,
        Timestamp('2025-11-26 00:00:00')],
       ['09988', 154.0, 154.9000091553, ..., 10330, 15705985024.0,
        Timestamp('2025-11-27 00:00:00')],
       ['09988', 151.8000030518, 153.200012207, ..., 6657, 10111444992.0,
        Timestamp('2025-11-28 00:00:00')]], shape=(1453, 8), dtype=object)

In [2]:
stock_list = fetch_stock_list()
assert len(stock_list) > 0
stock_name_dict = stock_list['name'].to_dict()
stock_list.head()


,code,volunit,decimal_point,name,pre_close,sse,sec
code,,,,,,,
000001,000001,100,2,平安银行,952.0025,sz,stock_cn
000002,000002,100,2,万 科Ａ,6.0600,sz,stock_cn
000004,000004,100,2,*ST国华,648.0400,sz,stock_cn
000006,000006,100,2,深振业Ａ,648.0000,sz,stock_cn
000007,000007,100,2,全新好,392.0000,sz,stock_cn


In [ ]:
from pymongo import ASCENDING
db = get_db()
database = db.get_collection('dk_data')
database.create_index([('code', ASCENDING)])
database.create_index([('datetime', ASCENDING)])


'datetime_1'

In [4]:
hkcode_list = fetch_get_hkstock_list('tdx')
assert len(hkcode_list) > 0
name_code_dict = hkcode_list.set_index('name')['code'].to_dict()
hkcode_list.head()


,code,name
code,,
00003,00003,00003
00005,00005,00005
00020,00020,00020
00027,00027,00027
00029,00029,00029


In [ ]:
root_path = r"D:\zxxq_xlsx"
file_names = os.listdir(root_path)
file_names = np.array(file_names)
file_names = file_names[[ True if 'V】奇衡DK全球视野学习材料' in file_names[i] else False for i in range(len(file_names))]][::-1]
for file_name in file_names[:20]:
    s_dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=None)
    sheet_names = np.array(list(s_dk_data.keys()))
    total_dk_data = []
    for sheet_name in sheet_names:
        dk_data = s_dk_data[sheet_name]
        dk_data.columns = ['code','name','date','R_value','L_value']
        if dk_data['code'].dtype == 'int64':
            dk_data['code'] = dk_data['code'].apply(lambda x : "0"*(5-len(str(x))) +  str(x))
        dk_data = dk_data[['code','R_value','L_value']]
        total_dk_data.append(dk_data)
    total_dk_data = pd.concat(total_dk_data)
    total_dk_data['datetime'] = str(pd.to_datetime(file_name.split(".")[0].split("-")[0][-8:]))[:10]
    insert_data_json = [i.to_dict() for _,i in total_dk_data.iterrows()]
    database.insert_many(insert_data_json)
    print("sucess",file_name)


sucess 【V】奇衡DK全球视野学习材料20251205.xlsx
sucess 【V】奇衡DK全球视野学习材料20251204.xlsx
sucess 【V】奇衡DK全球视野学习材料20251203.xlsx
sucess 【V】奇衡DK全球视野学习材料20251202.xlsx
sucess 【V】奇衡DK全球视野学习材料20251201.xlsx
sucess 【V】奇衡DK全球视野学习材料20251128.xlsx
sucess 【V】奇衡DK全球视野学习材料20251127.xlsx
sucess 【V】奇衡DK全球视野学习材料20251126.xlsx
sucess 【V】奇衡DK全球视野学习材料20251125.xlsx
sucess 【V】奇衡DK全球视野学习材料20251124.xlsx
sucess 【V】奇衡DK全球视野学习材料20251121.xlsx
sucess 【V】奇衡DK全球视野学习材料20251120.xlsx
sucess 【V】奇衡DK全球视野学习材料20251119.xlsx
sucess 【V】奇衡DK全球视野学习材料20251118.xlsx
sucess 【V】奇衡DK全球视野学习材料20251117.xlsx
sucess 【V】奇衡DK全球视野学习材料20251114.xlsx
sucess 【V】奇衡DK全球视野学习材料20251113.xlsx
sucess 【V】奇衡DK全球视野学习材料20251112.xlsx
sucess 【V】奇衡DK全球视野学习材料20251111.xlsx
sucess 【V】奇衡DK全球视野学习材料20251111-1.xlsx


In [101]:
# Fetch stock adjustment factors (subset to keep runtime reasonable)
codes = list(stock_list.index[:200])
data_fq = fetch_stock_adj(codes, '2023-01-01', '2025-01-01', format='pd')
assert data_fq is not None and len(data_fq) > 0
data_fq.head()
index_info = fetch_index_list()
future_list = fetch_future_list()
stock_list = fetch_stock_list()
etf_list = fetch_etf_list()

In [ ]:
root_path = r"D:\zxxq_xlsx"
file_names = os.listdir(root_path)
file_names = np.array(file_names)
file_names = file_names[[ True if 'V】奇衡DK星球学习材料' in file_names[i] else False for i in range(len(file_names))]]
for file_name in file_names[-1:]:

    print(file_name)
    date_time = file_name.split(".")[0][-1:]
    total_dk_data = []
    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=None)
    sheet_names = np.array(list(dk_data.keys()))
    print(sheet_names)
    sheet_name1 = sheet_names[[True if '矩形' in x else False  for x in sheet_names]][0]
    sheet_name2 = sheet_names[[True if 'UBW' in x else False  for x in sheet_names]][0]
    sheet_name3 = sheet_names[[True if '原共享' in x else False  for x in sheet_names]][0]
    sheet_name4 = sheet_names[[True if '主连' in x else False  for x in sheet_names]][0]
    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name1)
    dk_data.columns = dk_data.iloc[0]
    dk_data=dk_data.iloc[1:]
    dk_data['code'] = dk_data['NO.'].apply(lambda x : "0"*(6-len(str(x))) +  str(x))
    dk_data=dk_data.iloc[:,2:]
    dk_columns = ['L_value','R_value','R_switch','L_switch','L_label','R_label','R_bias','L_bias','acitve1','acitve2','exceed1','exceed2','exceed3','switch1','switch2','switch3','code']
    dk_data.columns = dk_columns
    total_dk_data.append(dk_data)

    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name2)
    dk_data.columns = dk_data.iloc[0]
    dk_data=dk_data.iloc[1:]
    dk_data['code'] = dk_data['NO.'].apply(lambda x : "0"*(6-len(str(x))) +  str(x))
    dk_data=dk_data.iloc[:,2:]
    dk_columns = ['L_value','R_value','R_switch','L_switch','L_label','R_label','R_bias','L_bias','acitve1','acitve2','exceed1','exceed2','exceed3','switch1','switch2','switch3','code']
    dk_data.columns = dk_columns
    total_dk_data.append(dk_data)

    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name3)
    dk_data.columns = dk_data.iloc[0]
    dk_data=dk_data.iloc[1:]
    dk_data['code'] = dk_data['NO.'].apply(lambda x : "0"*(6-len(str(x))) +  str(x))
    dk_data=dk_data.iloc[:,2:]
    dk_columns = ['L_value','R_value','R_switch','L_switch','L_label','R_label','R_bias','L_bias','acitve1','acitve2','exceed1','exceed2','exceed3','switch1','switch2','switch3','code']
    dk_data.columns = dk_columns
    total_dk_data.append(dk_data)


    sheet_name5 = sheet_names[[True if '情绪数据库' in x else False  for x in sheet_names]][0]
    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name5)
    dk_data = dk_data.iloc[5:,[0,2,3]]
    dk_data.columns = ['code','L_value','R_value']
    dk_data['code'] = dk_data['code'].apply(lambda x : "0"*(6-len(str(x))) +  str(x))
    total_dk_data.append(dk_data)

    if len(sheet_names[[True if '2023' in x else False  for x in sheet_names]]) > 0:
        sheet_name6 = sheet_names[[True if '2023' in x else False  for x in sheet_names]][0]
        dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name6)
        dk_data = dk_data.iloc[5:,[0,2,3]]
        dk_data.columns = ['code','L_value','R_value']
        dk_data['code'] = dk_data['code'].apply(lambda x : "0"*(6-len(str(x))) +  str(x))
        total_dk_data.append(dk_data)

    dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name4)
    dk_data.columns = dk_data.iloc[0]
    dk_data=dk_data.iloc[1:]
    dk_data['code']=dk_data.iloc[:,0].apply(lambda x : str(x)[:-2])
    dk_data=dk_data.iloc[:,2:]
    dk_columns = ['L_value','R_value','R_switch','L_switch','L_label','R_label','R_bias','L_bias','acitve1','acitve2','exceed1','exceed2','exceed3','switch1','switch2','switch3','code']
    dk_data.columns = dk_columns
    total_dk_data.append(dk_data)

    if '指数数据库' in sheet_names:
        sheet_name5 = sheet_names[[True if '指数数据库' in x else False  for x in sheet_names]][0]
        dk_data = pd.read_excel(r"%s\%s"%(root_path,file_name),sheet_name=sheet_name5)
        dk_data.columns = dk_data.iloc[0]
        dk_data=dk_data.iloc[1:]
        dk_data['code'] = dk_data['NO.'].apply(lambda x : "9" + "0"*(6-len(str(x))) +  str(x))
        dk_data=dk_data.iloc[:,2:]
        dk_columns = ['L_value','R_value','R_switch','L_switch','L_label','R_label','R_bias','L_bias','acitve1','acitve2','exceed1','exceed2','exceed3','switch1','switch2','switch3','code']
        dk_data.columns = dk_columns
        total_dk_data.append(dk_data)
    total_dk_data = pd.concat(total_dk_data)
    total_dk_data['datetime'] = file_name.split(".")[0][-8:]
    total_dk_data = total_dk_data.fillna(0)
    total_dk_data = total_dk_data.drop_duplicates("code")
    date_time = str(pd.to_datetime(file_name.split(".")[0][-8:]))[:10]
    fq_dict = data_fq[data_fq['date']==date_time].set_index("code")['adj'].to_dict()
    total_dk_data['adj'] = total_dk_data['code'].apply(lambda x : fq_dict[x] if x in fq_dict.keys() else 1)
    total_dk_data['L_value_adj'] = total_dk_data.apply(lambda x : x['adj']*x['L_value'],axis=1)
    total_dk_data['R_value_adj'] = total_dk_data.apply(lambda x : x['adj']*x['R_value'],axis=1)
    insert_data_json = [i.to_dict() for _,i in total_dk_data.iterrows()]

【V】奇衡DK星球学习材料20251205.xlsx
['原共享数据库' '矩形数据库' 'UBW数据库' '指数数据库' '情绪数据库' '主连数据库']


In [96]:
import re
import numpy as np
import pandas as pd


# =========================
# 1) Normalize helpers
# =========================
def _normalize_code_base(x) -> str:
    """
    规范成底层 code_base：
    - 含英文字母：原样（期货等）
    - 纯数字：
        * '1'/'1.0' -> '000001'
        * '000001' -> '000001'
        * 5位如 '00003'（港股常见）保持 5 位
        * 其它 <6 位补到 6 位
    """
    if x is None:
        return ""
    if isinstance(x, float) and np.isnan(x):
        return ""

    s = str(x).strip()
    s = re.sub(r"\.0$", "", s)

    if re.search(r"[A-Za-z]", s):
        return s

    if re.fullmatch(r"\d+", s):
        if len(s) == 5:
            return s
        if len(s) < 6:
            return s.zfill(6)
        return s

    return s


def tb_symbol_stock_etf(code_base: str) -> str:
    """
    A股/ETF 常规补后缀（不含 LOF/REITs）：
    - 60/68 + SH
    - 00/30/15 + SZ
    - 50/51/56/58 + SH
    """
    x = str(code_base)
    if not re.fullmatch(r"\d{6}", x):
        return x
    if x[:2] in ["60", "68", "50", "51", "56", "58"]:
        return x + ".SH"
    if x[:2] in ["00", "30", "15"]:
        return x + ".SZ"
    return x


# =========================
# 2) Enrich: code1/type
# =========================
def enrich_code1_and_type(
    total_dk_data: pd.DataFrame,
    index_info: pd.DataFrame,
    *,
    code_col: str = "code",
    rvalue_col: str = "R_value",
    datetime_col: str = "datetime",
    index_code_col: str = "code",
    index_preclose_col: str = "pre_close",
    index_sse_col: str = "sse",
    index_sec_col: str = "sec",
    # 仅用于冲突码（像 000001）判别指数
    rel_tol: float = 0.20,
    abs_tol: float = 200.0,
) -> pd.DataFrame:
    """
    输出新增两列：
      - code1: 加 .SH/.SZ（期货先保持原 code）
      - type : index / stock / etf / lof / reits / future / ''

    补充规则：
      - 16xxxxxx: LOF -> .SZ, type='lof'
      - 18xxxxxx: REITs -> .SH, type='reits'
      - 期货：含字母 -> type='future'（code1 先不改，后续用 patch_future_code1_to_main 统一映射主连）
    """
    # 防御：去掉重复列名
    df0 = total_dk_data.copy()
    df0 = df0.loc[:, ~df0.columns.duplicated()]
    orig_index = df0.index
    df = df0.reset_index(drop=True)

    # 防御：避免外部已有 _code_base
    df = df.drop(columns=["_code_base"], errors="ignore")
    df["_code_base"] = df[code_col].map(_normalize_code_base)

    # --- index_info 预处理（尽量只保留 index_cn） ---
    idx = index_info.copy()
    idx = idx.loc[:, ~idx.columns.duplicated()]
    if index_code_col not in idx.columns:
        idx = idx.reset_index().rename(columns={"index": index_code_col})
    idx = idx.drop(columns=["_code_base"], errors="ignore")
    idx["_code_base"] = idx[index_code_col].map(_normalize_code_base)

    if index_sec_col in idx.columns:
        idx = idx[idx[index_sec_col].astype(str).str.contains("index", case=False, na=False)].copy()

    keep_cols = ["_code_base"]
    if index_preclose_col in idx.columns:
        keep_cols.append(index_preclose_col)
    if index_sse_col in idx.columns:
        keep_cols.append(index_sse_col)
    idx = idx[keep_cols].drop_duplicates("_code_base", keep="first") if not idx.empty else idx

    # --- init ---
    df["code1"] = df["_code_base"].astype(str)
    df["type"] = ""

    base = df["_code_base"].astype(str)
    has_alpha = base.str.contains(r"[A-Za-z]", na=False)

    # future（先标类型，code1 暂不处理）
    df.loc[has_alpha, "type"] = "future"
    non_fut = ~has_alpha

    # LOF / REITs
    is_6d = base.str.fullmatch(r"\d{6}", na=False)
    lof_mask = non_fut & is_6d & base.str.startswith("16")
    reit_mask = non_fut & is_6d & base.str.startswith("18")

    df.loc[lof_mask, "type"] = "lof"
    df.loc[lof_mask, "code1"] = base.loc[lof_mask].values + ".SZ"

    df.loc[reit_mask, "type"] = "reits"
    df.loc[reit_mask, "code1"] = base.loc[reit_mask].values + ".SH"

    # --- merge index info ---
    if not idx.empty and (index_preclose_col in idx.columns):
        df2 = df.merge(idx, on="_code_base", how="left")
        in_index = non_fut & df2[index_preclose_col].notna()
    else:
        df2 = df.copy()
        in_index = pd.Series(False, index=df2.index)

    # 冲突前缀（容易与股票/ETF 重名）
    ambig_prefixes = ["60", "00", "30", "15", "68", "50", "51", "56", "58"]
    ambig = base.str[:2].isin(ambig_prefixes)

    grp_cols = [datetime_col, "_code_base"] if datetime_col in df2.columns else ["_code_base"]

    # 只允许覆盖空/stock/etf（不覆盖 lof/reits/future）
    eligible_for_index = df2["type"].isin(["", "stock", "etf"])

    # 1) 重复组：冲突码挑最接近 pre_close 的一条为 index
    dup_mask = df2.duplicated(subset=grp_cols, keep=False)
    cand_mask = in_index & ambig & dup_mask & eligible_for_index
    cand_cols = list(dict.fromkeys(grp_cols + [rvalue_col, index_preclose_col]))
    cand = df2.loc[cand_mask, cand_cols].copy()
    cand = cand.loc[:, ~cand.columns.duplicated()]

    if not cand.empty:
        cand["r"] = pd.to_numeric(cand[rvalue_col], errors="coerce")
        cand["pre"] = pd.to_numeric(cand[index_preclose_col], errors="coerce")
        cand = cand.dropna(subset=["r", "pre"])
        if not cand.empty:
            cand["diff"] = (cand["r"] - cand["pre"]).abs()
            idx_min = cand.groupby(grp_cols, sort=False)["diff"].idxmin()
            df2.loc[idx_min.to_numpy(), "type"] = "index"

    # 2) 非冲突指数码：在 index_info 就直接认 index（399xxx 等）
    non_ambig_index = in_index & (~ambig) & df2["type"].ne("index") & df2["type"].isin(["", "stock", "etf"])
    df2.loc[non_ambig_index, "type"] = "index"

    # 3) 冲突码剩余：阈值判别
    remain_ambig = in_index & ambig & df2["type"].ne("index") & df2["type"].isin(["", "stock", "etf"])
    if remain_ambig.any():
        r = pd.to_numeric(df2.loc[remain_ambig, rvalue_col], errors="coerce")
        pre = pd.to_numeric(df2.loc[remain_ambig, index_preclose_col], errors="coerce")
        diff = (r - pre).abs()
        thr = np.maximum(abs_tol, rel_tol * pre.abs())
        close = pre.notna() & r.notna() & (diff <= thr)
        df2.loc[close[close].index, "type"] = "index"

    # 4) index 补后缀
    is_index_mask = df2["type"].eq("index")
    if is_index_mask.any():
        if index_sse_col in df2.columns:
            sse = df2.loc[is_index_mask, index_sse_col].fillna("sh").astype(str).str.lower()
        else:
            sse = pd.Series("sh", index=df2.loc[is_index_mask].index)
        suffix = np.where(sse.eq("sz"), ".SZ", ".SH")
        df2.loc[is_index_mask, "code1"] = df2.loc[is_index_mask, "_code_base"].astype(str).to_numpy() + suffix

    # 5) 其余走 stock/etf
    remain = df2["type"].eq("") & non_fut & is_6d
    remain_base = df2.loc[remain, "_code_base"].astype(str)
    if not remain_base.empty:
        df2.loc[remain_base.index, "code1"] = remain_base.map(tb_symbol_stock_etf).values
        p2 = remain_base.str[:2]
        stock_rows = p2[p2.isin(["60", "00", "30", "15", "68"])].index
        etf_rows = p2[p2.isin(["50", "51", "56", "58"])].index
        df2.loc[stock_rows, "type"] = "stock"
        df2.loc[etf_rows, "type"] = "etf"

    out = df2[df.columns].copy()
    out = out.drop(columns=["_code_base"], errors="ignore")
    out.index = orig_index
    return out


# =========================
# 3) FUTURE patch: map to 主连（且明确“不用带 '-' 的”）
# =========================
def build_future_main_map_no_dash(
    future_list: pd.DataFrame,
    *,
    name_col: str = "name",
    keyword: str = "主连",
) -> dict:
    """
    构建 base(字母前缀) -> 主连symbol，且：
      - 过滤掉所有带 '-' 的主连（如 'L-FL8', 'PP-FL8', 'V-FL8'）
      - 只保留不带 '-' 的主连（如 'LL8', 'PPL8', 'VL8'）

    base 的提取：
      - '*L8' -> base=symbol[:-2]  (NIL8->NI, TL8->T, TLL8->TL)
    """
    fl = future_list.copy()

    # 只取 name 含“主连”的
    if name_col in fl.columns:
        fl = fl[fl[name_col].astype(str).str.contains(keyword, na=False)].copy()

    symbols = fl.index.astype(str)

    # 过滤：不要带 '-' 的
    symbols = [s for s in symbols.tolist() if "-" not in s]

    mp = {}
    for sym in symbols:
        sym = str(sym)
        if not sym.endswith("L8"):
            continue
        base = sym[:-2].upper()
        # 若出现重复 base（罕见），保留更短的
        if base not in mp or len(sym) < len(mp[base]):
            mp[base] = sym
    return mp


def patch_future_code1_to_main_no_dash(
    df: pd.DataFrame,
    future_list: pd.DataFrame,
    *,
    code_col: str = "code",
    type_col: str = "type",
    code1_col: str = "code1",
    name_col: str = "name",
    keyword: str = "主连",
) -> pd.DataFrame:
    """
    把 df 中 type=='future' 的 code1 改成“不带 '-' 的主连”symbol：
      - code='NI'    -> code1='NIL8'
      - code='SC00'  -> code1='SCL8'（取字母前缀 SC）
      - code='ZS2007'-> code1='ZSL8'（若存在）
    """
    out = df.copy()
    mp = build_future_main_map_no_dash(future_list, name_col=name_col, keyword=keyword)

    fut_mask = out[type_col].eq("future")
    if not fut_mask.any():
        return out

    codes = out.loc[fut_mask, code_col].astype(str).str.strip()

    # 取字母前缀（不是前两位）：SC00->SC, ZS2007->ZS
    base = codes.str.extract(r"^([A-Za-z]+)", expand=False).fillna("").str.upper()

    mapped = base.map(mp)  # Series: str 或 NaN

    # 用 np.where 覆盖（最稳，不会触发 len mismatch）
    out.loc[fut_mask, code1_col] = np.where(
        mapped.notna(),
        mapped.astype(str).to_numpy(),
        out.loc[fut_mask, code1_col].astype(str).to_numpy()
    )
    return out


# =========================
# 4) One-shot usage
# =========================
total_dk_data2 = enrich_code1_and_type(total_dk_data, index_info)
future_list = fetch_future_list()
total_dk_data2 = patch_future_code1_to_main_no_dash(total_dk_data2, future_list)

# check
total_dk_data2.query("type=='future'")[["code","code1"]].drop_duplicates().head(200)
total_dk_data2.query("type in ['lof','reits','index']")[["code","code1","type"]].head(50)


,code,code1,type
69,000683,000683.SH,index
76,000935,000935.SH,index
517,000007,000007.SH,index
519,000078,000078.SH,index
542,000697,000697.SH,index
558,000933,000933.SH,index
1063,000002,000002.SH,index
1066,000026,000026.SH,index
1067,000032,000032.SH,index
1071,000048,000048.SH,index


In [ ]:
import os
import re
import numpy as np
import pandas as pd


def read_first_sheet_strength_table(xlsx_path: str) -> pd.DataFrame:
    """
    Read the 1st sheet (sheet_name=0) and convert it into ONE dataframe with ENGLISH columns:
      - code              (代码)
      - short_name        (简称)
      - total_market_cap  (总市值)
      - type              (类型): weak / strong
          weak   = 强转弱
          strong = 弱转强
      - trade_date        (从文件名提取 8 位日期，如 20241009)

    Notes:
    - The sheet layout is two blocks side-by-side:
        [京沪深强转弱] 代码 简称 总市值 | [京沪深弱转强] 代码 简称 总市值
      and real data starts from row 3 (0-based row index 2).
    """
    # -------- trade_date from file name --------
    fname = os.path.basename(xlsx_path)
    dates = re.findall(r"(\d{8})", fname)          # 抓所有 8 位数字
    trade_date = dates[-1] if dates else None      # 取最后一个最稳（通常就在文件尾部）

    raw = pd.read_excel(xlsx_path, sheet_name=0, header=None, engine="openpyxl")

    # Left block: 强转弱 -> weak
    left = raw.iloc[2:, 0:3].copy()
    left.columns = ["code", "short_name", "total_market_cap"]
    left["type"] = "weak"

    # Right block: 弱转强 -> strong
    right = raw.iloc[2:, 3:6].copy()
    right.columns = ["code", "short_name", "total_market_cap"]
    right["type"] = "strong"

    df = pd.concat([left, right], ignore_index=True)

    # --- clean ---
    df = df.dropna(subset=["code"])

    # keep code as string, keep leading zeros
    df["code"] = (
        df["code"].astype(str)
        .str.strip()
        .str.replace(r"\.0$", "", regex=True)
    )
    df["code"] = df["code"].apply(lambda s: s.zfill(6) if s.isdigit() and len(s) < 6 else s)

    df["short_name"] = df["short_name"].astype(str).str.strip()

    # market cap might contain '--'
    df["total_market_cap"] = (
        df["total_market_cap"].astype(str).str.strip()
        .replace({"--": pd.NA, "nan": pd.NA, "None": pd.NA})
    )

    # add trade_date
    df["trade_date"] = trade_date  # 字符串 'YYYYMMDD'；若没抓到就是 None

    return df[["trade_date", "code", "short_name", "total_market_cap", "type"]]


# ===== Example =====
file_path = r"D:\zxxq_xlsx\奇衡DK星球指数学习材料20241009.xlsx"
strength_df = read_first_sheet_strength_table(file_path)
strength_df

,trade_date,code,short_name,total_market_cap,type
0,20241009,399101,中小综指,112811.38亿,weak
1,20241009,880551,大基金持股,18396.42亿,weak
2,20241009,880760,预制菜,15626.07亿,weak
3,20241009,880360,农林牧渔,10958.62亿,weak
4,20241009,880897,即将解禁,10173.52亿,weak
...,...,...,...,...,...
163,20241009,880843,业绩预降,10.73亿,strong
164,20241009,399438,绿色电力,NaN,strong
165,20241009,399366,能源金属,NaN,strong
166,20241009,399319,资源优势,NaN,strong


In [117]:
import os
import re
import numpy as np
import pandas as pd


def extract_trade_date_from_path(xlsx_path: str) -> str | None:
    """
    从文件名提取 8 位日期作为 trade_date。
    例如: 【V】奇衡DK全球视野学习材料20250801.xlsx -> '20250801'
    """
    fname = os.path.basename(xlsx_path)
    dates = re.findall(r"(\d{8})", fname)
    return dates[-1] if dates else None


def _find_header_row(raw: pd.DataFrame, max_scan: int = 30) -> int:
    """
    在前 max_scan 行里找表头行：至少包含 NO/Name/R/L（忽略大小写与末尾点号）
    """
    need = {"no", "name", "r", "l"}

    def norm(x) -> str:
        if pd.isna(x):
            return ""
        s = str(x).strip().lower()
        s = re.sub(r"\.+$", "", s)  # 去掉末尾 ...
        return s

    for i in range(min(max_scan, len(raw))):
        row = set(filter(None, (norm(v) for v in raw.iloc[i].tolist())))
        if need.issubset(row):
            return i
    return 0


def _read_one_sheet_global_view(
    xlsx_path: str,
    sheet_name: str,
    trade_date: str | None,
    hk_code_width: int = 5,   # sheet '1'（港股）代码补 5 位，如 3 -> 00003
) -> pd.DataFrame:
    """
    读取单个 sheet（结构: NO/Name/Date/R/L），丢弃 Date，返回英文列：
      trade_date, code, name, R, L
    """
    raw = pd.read_excel(xlsx_path, sheet_name=sheet_name, header=None, engine="openpyxl")
    hdr = _find_header_row(raw)

    # 取表头并标准化（去末尾点号）
    header_vals = [("" if pd.isna(x) else str(x).strip()) for x in raw.iloc[hdr].tolist()]
    cols = [re.sub(r"\.+$", "", c).strip() for c in header_vals]

    data = raw.iloc[hdr + 1 :].copy()
    data.columns = cols[: data.shape[1]]

    # 统一列名（忽略 NO/NO. Name/Name. Date/Date.）
    rename = {}
    for c in data.columns:
        cn = re.sub(r"\.+$", "", str(c)).strip().lower()
        if cn == "no":
            rename[c] = "code"
        elif cn == "name":
            rename[c] = "name"
        elif cn == "r":
            rename[c] = "R"
        elif cn == "l":
            rename[c] = "L"
        elif cn == "date":
            rename[c] = "_drop_date"

    data = data.rename(columns=rename)

    # 必要列检查
    need_cols = ["code", "name", "R", "L"]
    missing = [c for c in need_cols if c not in data.columns]
    if missing:
        raise ValueError(f"sheet={sheet_name} missing columns {missing}, got={list(data.columns)}")

    data = data[need_cols].copy()
    data = data.dropna(subset=["code"])

    # code 清洗：去掉 .0，转字符串
    code = (
        data["code"].astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
    )

    # sheet '1'：港股 NO 多为数字，补 5 位（00003/00005/00175...）
    if str(sheet_name) == "1":
        code = code.apply(lambda s: s.zfill(hk_code_width) if s.isdigit() and len(s) <= hk_code_width else s)

    data["code"] = code
    data["name"] = data["name"].astype(str).str.strip()
    data["R"] = pd.to_numeric(data["R"], errors="coerce")
    data["L"] = pd.to_numeric(data["L"], errors="coerce")

    data["trade_date"] = trade_date
    return data[["trade_date", "code", "name", "R", "L"]]


def read_global_view_file(
    xlsx_path: str,
    sheets: tuple[str, ...] = ("1", "2", "3"),
) -> pd.DataFrame:
    """
    读取 sheet '1','2','3'（或你传入的 sheets），合并成一个 dataframe。
    丢弃 Date 列，新增 trade_date（从文件名提取 8 位日期）。
    """
    trade_date = extract_trade_date_from_path(xlsx_path)
    dfs = [_read_one_sheet_global_view(xlsx_path, s, trade_date) for s in sheets]
    return pd.concat(dfs, ignore_index=True)


# ======================
# Example
# ======================
file_path = r"D:\zxxq_xlsx\【V】奇衡DK全球视野学习材料20250801.xlsx"
df = read_global_view_file(file_path, sheets=("1","2","3"))
print(df.head())
print(df.tail())


  trade_date   code    name      R      L
0   20250801  00003  香港中华煤气   7.29   6.39
1   20250801  00005    汇丰控股  96.25  76.62
2   20250801  00020    商汤-W   1.72   1.29
3   20250801  00029    达力集团  14.58  11.36
4   20250801  00101    恒隆地产   7.42   5.91
    trade_date  code                  name      R      L
206   20250801   QQQ  纳斯达克100ETF-ProShares  544.7  427.7
207   20250801   SPY     投资组合标普500ETF-SPDR  613.0  505.5
208   20250801   TLT   美国国债20年+ETF-iShares   90.8   82.5
209   20250801  SOXX   半导体ETF-iShares PHLX  234.2  160.5
210   20250801   VXX   恐慌指数期货短期合约ETN-iPath   81.8   45.1
